In [39]:
import os
import torch
from zkstats.core import (
    prover_gen_settings,
    prover_gen_proof,
    verifier_setup,
    verifier_verify,
)
from zkstats.computation import computation_to_model, State

cwd = os.getcwd()

# FIXME: fill this in with the path to your data
data_path = f"{cwd}/data.json"

# Paths to the output files
output_dir = f"{cwd}/out"
os.makedirs(output_dir, exist_ok=True)
model_onnx_path = f"{output_dir}/model.onnx"
compiled_model_path = f"{output_dir}/model.compiled"

pk_path = f"{output_dir}/model.pk"
vk_path = f"{output_dir}/model.vk"
proof_path = f"{output_dir}/model.pf"
settings_path = f"{output_dir}/settings.json"
witness_path = f"{output_dir}/witness.json"
comb_data_path = f"{output_dir}/comb_data.json"

## User-defined Computation

A computation should be of type `TComputation`. For example, the following code snippet defines a computation that computes the sum of the private data.

```python
def computation(state: State, x: list[torch.Tensor]):
    out_0 = state.median(x[0])
    out_1 = state.median(x[1])
    return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
```

FIXME: The following code snippet is entirely from the user. You MUST check
1. the code only performs zkstats-related operations.
2. the computation must not leak any information about the private data.

In [40]:
# This is just a dummy computation. Replace it with user's computation
# def computation(state: State, x: list[torch.Tensor]):
#     x_0 = x[0]
#     out_0 = state.median(x_0)
#     out_1 = state.median(x_0)
#     return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))

Generate settings and setup with user's computation.

In [41]:
_, model = computation_to_model(computation)
prover_gen_settings(
    [data_path],
    comb_data_path,
    model,
    model_onnx_path,
    "default",
    "resources",
    settings_path,
)
verifier_setup(model_onnx_path, compiled_model_path, settings_path, vk_path, pk_path)

/var/folders/t3/5psrvr1x0w1_6n9kx2n7d9700000gn/T/ipykernel_97835/3906390486.py:5: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/var/folders/t3/5psrvr1x0w1_6n9kx2n7d9700000gn/T/ipykernel_97835/3906390486.py:5: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))


==== Generate & Calibrate Setting ====


spawning module 0
spawning module 2


scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-25112,24986],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":12049,"total_const_size":1816,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}
==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 12.489933013916016 seconds


Generate proof with your data and user's computation.

In [42]:
prover_gen_proof(
    model_onnx_path,
    comb_data_path,
    witness_path,
    compiled_model_path,
    settings_path,
    proof_path,
    pk_path,
)


==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.55078125
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[3042937791208075219, 8157070662846698822, 3804781648660056856, 172406108020799675], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18341455175509539295, 12796101019039945164, 1607286914885633240, 1929881192315725821]]], 'proof': '01f9485e3484e27de162622863fdcf29cf81993a023d600b20173b37ca6d73392a2d6bee21aa3a00a12be030f57cf2fd67c3ba0182f1a879b53945993c2f20802eed1eeed24763f1d1070619ecbe97b47c3edab1d6f89a066dd6f6c6bfe0d3f60c26e2693859fb2be751d382d352ddb24570cba629a07bc55a58b33171825af1106e65e8bfb15161ac537b5a1463c993e8dbfb38dd3c410c64079587f6fe55c024f578ca70dd7700800e25cf651174e43fa3ddebd4e8036dbb4c8c2616fd2641084ea317141ecfab83be803e12254a766e35d526a9b6e1035d9c71b068ea8b4d125d355246396448552aaad9cd82c40568269f96acb66947c5d5b28c77331c420f816f2529b1b0d88b09c39a301c6665b04bdbeb406ffc42406017699ee7c2c42efc07fec604149bf8c787d91aca634ac1ae94b42655bcb480535ced9610e5311b28db6704f18b573cf2161321f75934a0e5ac7e51b1840543cbd4027471a391

Verify the proof to ensure it is correct

In [43]:
verifier_verify(proof_path, settings_path, vk_path)

!@# res:  True
prf instances:  [[[3042937791208075219, 8157070662846698822, 3804781648660056856, 172406108020799675], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18341455175509539295, 12796101019039945164, 1607286914885633240, 1929881192315725821]]]
num_inputs:  1
proof boolean:  1.0
proof result 0 : 49.55078125


Print the file paths. You should share the following files back to the user for them to verify the proof. You **SHOULD NOT** share more files otherwise data might be leaked.

In [44]:
model_onnx_path, settings_path, proof_path, vk_path
print("Model onnx:\t\t", model_onnx_path)
print("Settings:\t\t", settings_path)
print("Proof:\t\t\t", proof_path)
print("Verification key:\t", vk_path)

Model onnx:		 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/model.onnx
Settings:		 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/settings.json
Proof:			 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/model.pf
Verification key:	 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/model.vk
